In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import math
import pandas as pd
import re
import os

In [ ]:
def Reduce_CSV(path,filedesc):
  df=pd.read_csv(path)
  df['Label_Names'] = df.apply(lambda row: ', '.join([col for col in row.index[8:33] if row[col] == 1]), axis=1)
  single_valued=df[df['Label_Names'].str.contains('^[^,]+$', regex=True)]
  reduced_df=single_valued.copy()
  values = reduced_df['Label_Names'].unique()
  for value in values:
    if len(reduced_df[reduced_df['Label_Names'] == value]) > 2000:
      indices_to_drop = reduced_df[reduced_df['Label_Names'] == value].sample(n=len(reduced_df[reduced_df['Label_Names'] == value]) - 1000).index
      reduced_df.drop(indices_to_drop, inplace=True)
  reduced_df.to_csv(f'Reduced_Emotic_{filedesc}.csv',index=False)

In [ ]:
Reduce_CSV('/content/drive/MyDrive/Colab Notebooks/ML/ML Project/Emotic_Train.csv','Train')
Reduce_CSV('/content/drive/MyDrive/Colab Notebooks/ML/ML Project/Emotic_Test.csv','Test')
Reduce_CSV('/content/drive/MyDrive/Colab Notebooks/ML/ML Project/Emotic_Validate.csv','Validate')

In [ ]:
new_df=pd.read_csv('/content/Reduced_Emotic_Train.csv')

In [ ]:
new_df['Label_Names'].value_counts()

Anticipation       1097
Happiness          1066
Engagement         1000
Disconnection       811
Confidence          344
Excitement          264
Peace               203
Sensitivity         140
Pleasure            137
Doubt/Confusion     115
Fatigue             108
Sympathy             88
Surprise             85
Sadness              74
Affection            72
Annoyance            59
Anger                55
Esteem               52
Disquietment         52
Embarrassment        40
Disapproval          27
Fear                 26
Pain                 24
Aversion             19
Suffering            18
Name: Label_Names, dtype: int64

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML/ML Project/Emotic_Test.csv')

In [ ]:
df.head()

,File_Name,File_Path,Img_Height,Img_Width,X_Min,X_Max,Y_Min,Y_Max,Affection,Anger,...,Happiness,Pain,Peace,Pleasure,Sadness,Sensitivity,Suffering,Surprise,Sympathy,Yearning
0,011xa4rukq1oga5il9.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/emodb_...,300,450,246,342,90,260,0,0,...,1,1,0,0,0,0,1,1,0,1
1,011xa4rukq1oga5il9.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/emodb_...,300,450,174,241,161,263,1,0,...,1,0,0,0,0,0,0,0,0,0
2,01h2xzi1k8cthigbme.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/emodb_...,300,450,4,380,2,302,0,0,...,1,0,0,1,0,0,0,0,0,0
3,01qhe9raflmm17xf7k.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/emodb_...,513,684,45,291,85,512,0,0,...,1,0,0,0,0,0,0,0,0,0
4,01x5mmqg2sxy6yv2x4.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/emodb_...,300,590,153,423,35,263,0,0,...,1,0,1,1,0,0,0,0,0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7280 entries, 0 to 7279
Data columns (total 34 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   File_Name        7280 non-null   object
 1   File_Path        7280 non-null   object
 2   Img_Height       7280 non-null   int64 
 3   Img_Width        7280 non-null   int64 
 4   X_Min            7280 non-null   int64 
 5   X_Max            7280 non-null   int64 
 6   Y_Min            7280 non-null   int64 
 7   Y_Max            7280 non-null   int64 
 8   Affection        7280 non-null   int64 
 9   Anger            7280 non-null   int64 
 10  Annoyance        7280 non-null   int64 
 11  Anticipation     7280 non-null   int64 
 12  Aversion         7280 non-null   int64 
 13  Confidence       7280 non-null   int64 
 14  Disapproval      7280 non-null   int64 
 15  Disconnection    7280 non-null   int64 
 16  Disquietment     7280 non-null   int64 
 17  Doubt/Confusion  7280 non-null   

In [ ]:
df['Label_Names'] = df.apply(lambda row: ', '.join([col for col in row.index[8:33] if row[col] == 1]), axis=1)

In [ ]:
df.head()

,File_Name,File_Path,Img_Height,Img_Width,X_Min,X_Max,Y_Min,Y_Max,Affection,Anger,...,Pain,Peace,Pleasure,Sadness,Sensitivity,Suffering,Surprise,Sympathy,Yearning,Label_Names
0,011xa4rukq1oga5il9.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/emodb_...,300,450,246,342,90,260,0,0,...,1,0,0,0,0,1,1,0,1,"Disapproval, Doubt/Confusion, Fatigue, Happine..."
1,011xa4rukq1oga5il9.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/emodb_...,300,450,174,241,161,263,1,0,...,0,0,0,0,0,0,0,0,0,"Affection, Engagement, Happiness"
2,01h2xzi1k8cthigbme.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/emodb_...,300,450,4,380,2,302,0,0,...,0,0,1,0,0,0,0,0,0,"Happiness, Pleasure"
3,01qhe9raflmm17xf7k.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/emodb_...,513,684,45,291,85,512,0,0,...,0,0,0,0,0,0,0,0,0,"Disconnection, Excitement, Happiness"
4,01x5mmqg2sxy6yv2x4.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/emodb_...,300,590,153,423,35,263,0,0,...,0,1,1,0,0,0,0,0,0,"Confidence, Engagement, Happiness, Peace, Plea..."


In [ ]:
df['Label_Names'].nunique()

1088

In [ ]:
df['Label_Names'].value_counts()

Engagement                                                   977
Anticipation, Engagement                                     323
Happiness                                                    217
Anticipation                                                 198
Confidence, Engagement                                       172
                                                            ... 
Anticipation, Engagement, Sadness, Sympathy                    1
Affection, Anticipation, Engagement, Happiness, Peace          1
Anticipation, Confidence, Disquietment                         1
Affection, Anticipation, Esteem, Fear, Sympathy                1
Anticipation, Confidence, Excitement, Happiness, Surprise      1
Name: Label_Names, Length: 1088, dtype: int64

In [ ]:
#reduced_df=df.copy()

In [ ]:
# values = reduced_df['Label_Names'].unique()

# for value in values:
#     total_value_count = (reduced_df['Label_Names'] == value).sum()
#     if value == 'Engagement':
#       count_to_drop = int(0.75 * total_value_count)
#       indices_to_drop = reduced_df[reduced_df['Label_Names'] == value].sample(n=count_to_drop).index
#       reduced_df.drop(indices_to_drop, inplace=True)
#     elif total_value_count>=500:
#       count_to_drop = int(0.5 * total_value_count)
#       indices_to_drop = reduced_df[reduced_df['Label_Names'] == value].sample(n=count_to_drop).index
#       reduced_df.drop(indices_to_drop, inplace=True)
#     else:
#       continue

In [ ]:
single_valued=df[df['Label_Names'].str.contains('^[^,]+$', regex=True)]

In [ ]:
cat = ['Affection','Anger','Annoyance','Anticipation','Aversion','Confidence','Disapproval','Disconnection','Disquietment','Doubt/Confusion','Embarrassment','Engagement','Esteem','Excitement','Fatigue','Fear','Happiness','Pain','Peace','Pleasure','Sadness','Sensitivity','Suffering','Surprise','Sympathy','Yearning']

In [ ]:
single_valued['Label_Names'].value_counts()

Engagement         977
Happiness          217
Anticipation       198
Disconnection      161
Excitement          52
Confidence          47
Peace               31
Pleasure            29
Sensitivity         28
Sympathy            25
Doubt/Confusion     19
Fatigue             18
Esteem              16
Affection           16
Sadness             14
Anger               13
Disquietment        11
Annoyance            9
Surprise             9
Disapproval          7
Aversion             6
Fear                 5
Pain                 3
Embarrassment        2
Suffering            2
Name: Label_Names, dtype: int64

In [ ]:
unique_emotions = single_valued['Label_Names'].unique()

In [ ]:
set1 = set(cat)
set2 = set(unique_emotions)

set1.difference(set2)

{'Yearning'}

In [ ]:
single_valued['Label_Names'].nunique()

25

In [ ]:
reduced_df=single_valued.copy()
indices_to_drop = reduced_df[reduced_df['Label_Names'] == 'Engagement'].sample(n=4219).index
reduced_df.drop(indices_to_drop, inplace=True)

In [ ]:
reduced_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5976 entries, 2 to 23685
Data columns (total 35 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   File_Name        5976 non-null   object
 1   File_Path        5976 non-null   object
 2   Img_Height       5976 non-null   int64 
 3   Img_Width        5976 non-null   int64 
 4   X_Min            5976 non-null   int64 
 5   X_Max            5976 non-null   int64 
 6   Y_Min            5976 non-null   int64 
 7   Y_Max            5976 non-null   int64 
 8   Affection        5976 non-null   int64 
 9   Anger            5976 non-null   int64 
 10  Annoyance        5976 non-null   int64 
 11  Anticipation     5976 non-null   int64 
 12  Aversion         5976 non-null   int64 
 13  Confidence       5976 non-null   int64 
 14  Disapproval      5976 non-null   int64 
 15  Disconnection    5976 non-null   int64 
 16  Disquietment     5976 non-null   int64 
 17  Doubt/Confusion  5976 non-null  

In [ ]:
reduced_df['Label_Names'].value_counts()

Anticipation       1097
Happiness          1066
Engagement         1000
Disconnection       811
Confidence          344
Excitement          264
Peace               203
Sensitivity         140
Pleasure            137
Doubt/Confusion     115
Fatigue             108
Sympathy             88
Surprise             85
Sadness              74
Affection            72
Annoyance            59
Anger                55
Esteem               52
Disquietment         52
Embarrassment        40
Disapproval          27
Fear                 26
Pain                 24
Aversion             19
Suffering            18
Name: Label_Names, dtype: int64

In [ ]:
reduced_df.to_csv('Reduced_Emotic.csv',index=False)

In [ ]:
#xmin: , xmax: , ymin: , ymax: , [1]